In [156]:
class Row:
    def __init__(self, line):
        # The line below was a nasty bug! I missed the starting of a row if the
        # first point was a wall. I only managed to find the bug after I compared
        # the out put of # https://www.reddit.com/r/adventofcode/comments/zsnghh/2022_day_22_part_1_solution_works_only_for_the/
        # with my solution.

        #self.x0 = line.index('.') + 1 BUG
        wall_pos = line.index('#') if '#' in line else 1000_000
        self.x0 = min(line.index('.'), wall_pos) + 1
        self.x1 = len(line)
        self.line = line

    def is_valid_column(self, x):
        return self.x0 <= x <= self.x1

    def is_wall(self, x):
        return self.line[x-1] == "#"

    def right(self, x):
        xx = x + 1
        if xx > self.x1:
            xx = self.x0
        if self.is_wall(xx):
            return None
        else:
            return xx

    def left(self, x):
        xx = x - 1
        if xx < self.x0:
            xx = self.x1
        if self.is_wall(xx):
            return None
        else:
            return xx


class Board:
    def __init__(self, lines):
        self.rows = [Row(line.rstrip()) for line in lines]
        self.y0 = 1
        self.y1 = len(self.rows)

    def row(self, y):
        return self.rows[y-1]

    def start(self):
        row = self.row(self.y0)
        return (row.x0, self.y0)

    def forward(self, pos, orientation):
        x, y = pos
        if orientation == 0:
            return self.right(x, y)
        elif orientation == 1:
            return self.down(x, y)
        elif orientation == 2:
            return self.left(x, y)
        elif orientation == 3:
            return self.up(x, y)
        else:
            raise Exception("Wrong Orientation: %d" % orientation)
        
    def right(self, x, y):
        xx = self.row(y).right(x)
        if xx != None:
            return (xx, y)
        else:
            return None

    def left(self, x, y):
        xx = self.row(y).left(x)
        if xx != None:
            return (xx, y)
        else:
            return None

    def down(self, x, y):
        yy = y + 1
        if yy > self.y1:
            yy = self.y0
        row = self.row(yy)
        if not row.is_valid_column(x):
            # find upmost valid row for column x
            for yy in range(self.y0, self.y1+1):
                row = self.row(yy)
                if row.is_valid_column(x):
                    break
        if row.is_wall(x):
            return None
        else:
            return (x, yy)

    def up(self, x, y):
        yy = y - 1
        if yy < self.y0:
            yy = self.y1
        row = self.row(yy)
        if not row.is_valid_column(x):
            # find lowest valid row for column x
            for yy in range(self.y1, self.y0-1, -1):
                row = self.row(yy)
                if row.is_valid_column(x):
                    break
        if row.is_wall(x):
            return None
        else:
            return (x, yy)

Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^).

In [7]:
def move(board, pos, orientation, num_steps):
    for n in range(num_steps):
        fwd = board.forward(pos, orientation)
        if fwd is None:
            break
        else:
            pos = fwd
    return pos

### With the provided input

**That's not nice!**

In the exmaple we only have alternating turns: R,L,R,L,...  but
in the provided input we have turns like R,R,L,...  !

In [158]:
def read_input2(file_name):
    with open(file_name) as f:
        lines = [x.rstrip() for x in f.readlines()]
    board = Board(lines[:-2])
    instructions = lines[-1]

    res = []
    curr = ""
    for c in instructions:
        if c == "R":
            res.append(int(curr))        
            res.append("R")
            curr = ""
        elif c == "L":
            res.append(int(curr))        
            res.append("L")
            curr = ""
        else:
            curr += c
    res.append(int(curr))

    return board, res

In [159]:
board, moves = read_input2("example")

In [160]:
right_turn = {0: 1, 1: 2, 2: 3, 3: 0}
left_turn = {0: 3, 1: 0, 2: 1, 3: 2}

s = board.start()
orientation = 0
print(s)
for step in moves:
    if type(step) == int:
        s = move(board, s, orientation, step)
    else:
        if step == "R":
            orientation = right_turn[orientation]
        else:
            orientation = left_turn[orientation]

s

(9, 1)


(8, 6)

With the provided input

In [161]:
board, moves = read_input2("input")
s = board.start()
orientation = 0
print(s)
for step in moves:
    if type(step) == int:
        s = move(board, s, orientation, step)
    else:
        if step == "R":
            orientation = right_turn[orientation]
        else:
            orientation = left_turn[orientation]

(51, 1)


In [162]:
s, orientation

((107, 1), 0)

In [154]:
1000 * 1 + 4 * 107 + 0

1428

In [163]:
marker = {0: '>', 1: 'v', 2: '<', 3: '^'}

def mark_trail(board, pos, orientation):
    x, y = pos
    row = board.row(y)
    cs = [c for c in row.line]
    cs[x-1] = marker[orientation]
    row.line = "".join(cs)

def trail(board, pos, orientation, num_steps):
    mark_trail(board, pos, orientation)
    for n in range(num_steps):
        fwd = board.forward(pos, orientation)
        if fwd is None:
            break
        else:
            pos = fwd
            mark_trail(board, pos, orientation)

    return pos

In [ ]:
board, moves = read_input2("input")
s = board.start()
orientation = 0
def print_board(board):
    for row in board.rows:
        print(row.line)

for step in moves[:20]:
    print(step)
    if type(step) == int:
        s = trail(board, s, orientation, step)
    else:
        if step == "R":
            orientation = right_turn[orientation]
        else:
            orientation = left_turn[orientation]
        mark_trail(board, s, orientation)

    print_board(board)
    print()

# Part 2

**WARNING**  
This is not pretty! Everything is hard-wired. Don't try this at home!

My self-made cube for reference:

![alt text](cube.jpg "A 2d map of the cube from my input")

In [216]:
# hard-wired: each side is 50x50
# and hard-wired for my input

def which_side(x, y):
    if 50 < x <= 100:
        if y <= 50:
            return 6
        elif y <= 100:
            return 3
        else:
            return 1
    elif x > 100:
        return 2
    elif y <= 150:
        return 5
    else:
        return 4

In [217]:
board, moves = read_input2("input")

In [ ]:
for y in range(1, 201):
    for x in range(1, 151):
        row = board.row(y)
        if row.is_valid_column(x):
            print(which_side(x, y), end="")
        else:
            print(" ", end="")
    print()

In [173]:
class Edge:
    def __init__(self, face, func, new_orientation):
        # leading from face start to face end
        self.func = func
        self.face = face # never used
        self.new_orientation = new_orientation

    def cross(self, d):
        return (self.func(d), self.new_orientaion)
        
# orientations
RIGHT, DOWN, LEFT, UP = 0, 1, 2, 3

class Side:
    def __init__(self, x0, y0, right, down, left, up):
        self.x0 = x0
        self.y0 = y0
        self.right = right
        self.down = down
        self.left = left
        self.up = up

    def move_to(self, xx, yy, orientation):
        x = xx - self.x0
        y = yy - self.y0
        if 0 <= x < 50 and 0 <= y < 50:
            # inside this side, nothing to do
            return ((xx, yy), orientation)
        else:
            if x < 0:
                return self.left.cross(y)
            elif x >= 50:
                return self.right.cross(y)
            elif y < 0:
                return self.up.cross(x)
            else:
                return self.down.cross(x)


cube = {
    1: Side(51, 101, right=Edge(2, lambda d: (150, 50-d), LEFT), down=Edge(4, lambda d: (50, 151+d), LEFT), left=Edge(5, lambda d: ()))
}


In [272]:
class Board2(Board):
    # hard-wired: each side is 50x50 and hard-wired for my input
    #
    # Path are "<side from><side to><R: Turn right, 
    #                                L: Turn left, 
    #                                S: Staight ahead,
    #                                B: Backwards>"
    paths = ["14R46L63S31S",
             "15S56B62S21B",
             "32L24S45S53R"]

    # RDLU
    edges = {
        1: [2, 4, 5, 3],
        2: [1, 3, 6, 4],
        3: [2, 1, 5, 6],
        4: [1, 2, 6, 5],
        5: [1, 4, 6, 3],
        6: [2, 3, 5, 4]
    }
    
    def which_side(self, x, y):
        if 50 < x <= 100:
            if y <= 50:
                return 6
            elif y <= 100:
                return 3
            else:
                return 1
        elif x > 100:
            return 2
        elif y <= 150:
            return 5
        else:
            return 4

    def maybe_change_side(self, pos, orientation, start_side, dest_side):
        x, y = pos
        edge = "%d%d" % (start_side, dest_side)
        if edge == "21":
            return (self.forward((101, 151 - y), LEFT), LEFT)
        elif edge == "12":
            return (self.forward((151, 151 - y), LEFT), LEFT)
        elif edge == "56":
            return (self.forward((50, 151 - y), RIGHT), RIGHT)
        elif edge == "65":
            return (self.forward((0, 151 - y), RIGHT), RIGHT)
        elif edge == "14":
            return (self.forward((51, 100 + x), LEFT), LEFT)
        elif edge == "41":
            return (self.forward((y - 100, 151), UP), UP)
        elif edge == "53":
            return (self.forward((50, 50 + x), RIGHT), RIGHT)
        elif edge == "35":
            return (self.forward((y - 50, 100), DOWN), DOWN)
        elif edge == "46":
            return (self.forward((y - 100, 0), DOWN), DOWN)
        elif edge == "64":
            return (self.forward((0, x + 100), RIGHT), RIGHT)
        elif edge == "32":
            return (self.forward((y + 50, 51), UP), UP)
        elif edge == "23":
            return (self.forward((101, x - 50), LEFT), LEFT)
        elif edge == "24":
            return (self.forward((x - 100, 201), UP), UP)
        elif edge == "42":
            return (self.forward((x + 100, 0), DOWN), DOWN)

        else:
            raise Exception("Unknown change %s" % edge)

    def row(self, y):
        try:
            res = super().row(y)
        except IndexError:
            res = None
        return res

    def forward_with_orientation(self, pos, orientation):
        x, y = pos
        if orientation == 0:
            xx, yy = x + 1, y
        elif orientation == 1:
            xx, yy = x, y + 1
        elif orientation == 2:
            xx, yy = x - 1, y
        elif orientation == 3:
            xx, yy = x, y - 1
        else:
            raise Exception("Wrong Orientation: %d" % orientation)

        row = self.row(yy)
        if row and row.is_valid_column(xx):
            # we can move simply
            return (self.forward(pos, orientation), orientation)
        else:
            my_side = self.which_side(x, y)
            other_side = self.edges[my_side][orientation]
            return self.maybe_change_side(pos, orientation, my_side, other_side)

In [273]:
def move2(board, pos, orientation, num_steps):
    for n in range(num_steps):
        fwd, new_orientation = board.forward_with_orientation(pos, orientation)
        if fwd is None:
            break
        else:
            pos = fwd
            orientation = new_orientation
    return pos, orientation

In [274]:
def read_input2_2(file_name):
    with open(file_name) as f:
        lines = [x.rstrip() for x in f.readlines()]
    board = Board2(lines[:-2])
    instructions = lines[-1]

    res = []
    curr = ""
    for c in instructions:
        if c == "R":
            res.append(int(curr))        
            res.append("R")
            curr = ""
        elif c == "L":
            res.append(int(curr))        
            res.append("L")
            curr = ""
        else:
            curr += c
    res.append(int(curr))

    return board, res

In [275]:
def part2(file_name):
    board, moves = read_input2_2(file_name)
    pos = board.start()
    orientation = 0
    print(pos)
    for step in moves:
        if type(step) == int:
            pos, orientation = move2(board, pos, orientation, step)
        else:
            if step == "R":
                orientation = right_turn[orientation]
            else:
                orientation = left_turn[orientation]

    return pos, orientation

In [276]:
part2("input")

(51, 1)


((95, 142), 0)

In [277]:
1000* 142 + 4 * 95 + 0

142380

In [271]:
1000 * 192 + 4 * 30 + 1  ## still not right

192121

In [267]:
1000 * 130 + 4 * 39 + 1  # too low

130157

In [246]:
1000 * 49 + 4 * 141 + 2 # too low

49566

**With Trail**

In [247]:
marker = {0: '>', 1: 'v', 2: '<', 3: '^'}

def mark_trail(board, pos, orientation):
    x, y = pos
    row = board.row(y)
    cs = [c for c in row.line]
    cs[x-1] = marker[orientation]
    row.line = "".join(cs)

def trail2(board, pos, orientation, num_steps):
    mark_trail(board, pos, orientation)
    for n in range(num_steps):
        fwd, new_orientation = board.forward_with_orientation(pos, orientation)
        if fwd is None:
            break
        else:
            pos = fwd
            orientation = new_orientation
            mark_trail(board, pos, orientation)
    return pos, orientation

In [ ]:
board, moves = read_input2_2("input")
pos = board.start()
orientation = 0
print(pos)
for step in moves[:20]:
    print(step)
    if type(step) == int:
        pos, orientation = trail2(board, pos, orientation, step)
    else:
        if step == "R":
            orientation = right_turn[orientation]
        else:
            orientation = left_turn[orientation]
        mark_trail(board, pos, orientation)

    print_board(board)
    print()

In [211]:
paths = ["14R46L63S31S",
             "15S56B62S21B",
             "32L24S45S53R"]

In [215]:
x = 51
y = 100
(x-1) // 50, (y-1) // 50

(1, 1)